# LABORATORIO 3 - SIS 420 - Clasificación multiclase

## Introduction

En este ejercicio se implementa la regresion one-vs-all a un conjunto de datos en una empresa financiera global que evalua de puntuación crediticia para reducir los esfuerzos manuales y construir un modelo de aprendizaje automático que pueda clasificar el puntaje crediticio. (Bueno, Medio, Malo).

*   Nombre Dataset: **Credit score classification**
*   Url Dataset: https://www.kaggle.com/datasets/parisrohan/credit-score-classification



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para preprocesamiento del Dataset
import pandas as pd
# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [3]:
# Cargar el dataset
data = pd.read_csv('/content/drive/MyDrive/SIS 420 INTELIGENCIA ARTIFICAL 2024/MIS DATASETS/LAB 3/Credit score classification/train.csv', delimiter=',')


<ipython-input-3-c4deedc9d6b6>:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/SIS 420 INTELIGENCIA ARTIFICAL 2024/MIS DATASETS/LAB 3/Credit score classification/train.csv', delimiter=',')


In [4]:
dataset = data.copy()
# Quitamos: numero de seguro social(SSN), identificador(ID) y Customer_ID (Id Cliente) porque no aplica al modelo puesto que no sera un dato
# relavante que determine en nuestro caso
# Eliminar las columna con nombre 'SSN, ID, Customer_ID'
dataset.drop('SSN', axis=1, inplace=True)
dataset.drop('ID', axis=1, inplace=True)
dataset.drop('Customer_ID', axis=1, inplace=True)

In [5]:
# Aqui eliminamos los jemplos que cuenten con campos vacios en alguna de las caracteristicas del dataset.
data_sinf = dataset.dropna()

# Verificar el nuevo conteo de filas y columnas
print(data_sinf.shape)
print(data_sinf.dtypes)
#Tenemos listo el dataset de entrenamiento procesado con datos conscistentes

(53049, 25)
Month                        object
Name                         object
Age                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                 object
dtype: object


In [6]:
#Procedemos a convertir los datos de las caracteristicas a valores numericos para que podamos
#utilizar  de manera eficiente en el modelo mediante pandas
# Convertir columnas categóricas a numéricas (astype('category').cat.codes, convierte las categorías en valores numéricos únicos.)
# Luego, cat.codes extrae estos códigos numéricos asignados a cada categoría.
data_proc = data_sinf.copy()
data_proc['Month'] = data_proc['Month'].astype('category').cat.codes
data_proc['Name'] = data_proc['Name'].astype('category').cat.codes
data_proc['Age'] = data_proc['Age'].astype('category').cat.codes
data_proc['Occupation'] = data_proc['Occupation'].astype('category').cat.codes
data_proc['Annual_Income'] = data_proc['Annual_Income'].astype('category').cat.codes
data_proc['Num_of_Loan'] = data_proc['Num_of_Loan'].astype('category').cat.codes
data_proc['Type_of_Loan'] = data_proc['Type_of_Loan'].astype('category').cat.codes
data_proc['Num_of_Delayed_Payment'] = data_proc['Num_of_Delayed_Payment'].astype('category').cat.codes
data_proc['Changed_Credit_Limit'] = data_proc['Changed_Credit_Limit'].astype('category').cat.codes
data_proc['Credit_Mix'] = data_proc['Credit_Mix'].astype('category').cat.codes
data_proc['Outstanding_Debt'] = data_proc['Outstanding_Debt'].astype('category').cat.codes
data_proc['Credit_History_Age'] = data_proc['Credit_History_Age'].astype('category').cat.codes
data_proc['Payment_of_Min_Amount'] = data_proc['Payment_of_Min_Amount'].astype('category').cat.codes
data_proc['Amount_invested_monthly'] = data_proc['Amount_invested_monthly'].astype('category').cat.codes
data_proc['Payment_Behaviour'] = data_proc['Payment_Behaviour'].astype('category').cat.codes
data_proc['Monthly_Balance'] = data_proc['Monthly_Balance'].astype('category').cat.codes

# Definir el mapeo de clasificación para nuestra y a predecir asignandole los valores que deseamos
clasificacion = {'Good': 3, 'Standard': 2, 'Poor': 1}

# Aplicar el mapeo a la columna 'Credit_score'
data_proc['Credit_Score'] = data_proc['Credit_Score'].map(clasificacion)

# Verificar que las columnas categóricas se han convertido a numéricas
print(data_proc.head())
print(data_proc.dtypes)


    Month  Name  Age  Occupation  Annual_Income  Monthly_Inhand_Salary  \
0       3    75  184          12           4604            1824.843333   
6       4    75  184          12           4604            1824.843333   
8       3  6750  269          15           7905            3037.986667   
9       2  6750  257          13           7905            3037.986667   
10      6  6750  257          13           7906            3037.986667   

    Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  Num_of_Loan  ...  \
0                   3                4              3          134  ...   
6                   3                4              3          134  ...   
8                   2                4              6            1  ...   
9                   2                4              6            1  ...   
10                  2             1385              6            1  ...   

    Credit_Mix  Outstanding_Debt  Credit_Utilization_Ratio  \
0            3             10500          

In [7]:
#finalmente ordenamos el dataset de manera que la clasificacion del Credit_Score nuestra y a predecir este de manera agrupada y descendente
data_proc = data_proc.sort_values(by='Credit_Score')

# Verificar el DataFrame ordenado
print(data_proc.head())

       Month  Name  Age  Occupation  Annual_Income  Monthly_Inhand_Salary  \
99999      1  5903  215           9           8830            3359.415833   
73962      6  1510  315          15           2933           13006.396667   
31733      5   508  174           7           6588            2441.178333   
73967      1  1510  315           7           2933           13006.396667   
73978      6  5852  341           7           2666            1530.341667   

       Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  Num_of_Loan  ...  \
99999                  4                6              7           95  ...   
73962                  1                7              9          116  ...   
31733                  8                6             11          152  ...   
73967                  1                7              9           95  ...   
73978                  8               10             20          153  ...   

       Credit_Mix  Outstanding_Debt  Credit_Utilization_Ratio  \
999

In [8]:
# La entrada es de 25 caracteristicas + un elemento contando con x0
input_layer_size  = 26

#Etiquetas del Credit_Score (Good=3, Standart=2 y Poor=1)
num_labels = 3

X = data_proc.iloc[:, :-1]  # Selecciona todas las filas y todas las columnas excepto la última
y = data_proc.iloc[:, -1]    # Selecciona todas las filas y la última columna

y[y == 3] = 0
print(X)
print(y)

m = y.size

       Month  Name  Age  Occupation  Annual_Income  Monthly_Inhand_Salary  \
99999      1  5903  215           9           8830            3359.415833   
73962      6  1510  315          15           2933           13006.396667   
31733      5   508  174           7           6588            2441.178333   
73967      1  1510  315           7           2933           13006.396667   
73978      6  5852  341           7           2666            1530.341667   
...      ...   ...  ...         ...            ...                    ...   
37741      5   470  123          13           4015            1205.193333   
62535      1  8280  621          13          14519            6907.982009   
37740      7   470  123          13           4015            1205.193333   
87390      4  2722  602           1           5689            1320.708010   
0          3    75  184          12           4604            1824.843333   

       Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  Num_of_Loan  ... 

<ipython-input-8-8cdf38d83bcc>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == 3] = 0


In [9]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [10]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [11]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agraga el termino de intercepción a A
# X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
X = X_norm
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

In [12]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [13]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [14]:
def oneVsAll(X, y, num_labels, lambda_):
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 100}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [15]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(3, 25)


In [16]:
print(all_theta)

[[-2.34825987e+00 -3.10248140e-02 -7.23867576e-03  1.15722480e-01
  -3.24271798e-02 -3.91973940e-02  1.29712865e-01 -1.09817118e-02
  -1.68012713e-03  6.11672807e-03 -1.92109014e-01  3.06794793e-04
  -1.21752945e+00  8.66952390e-02  9.83181058e-02 -1.86097555e-02
  -3.10045790e-01  7.08245282e-02 -3.61456322e-02  1.29173624e-01
  -4.85342597e-01  9.79529718e-04  2.71282224e-02 -1.29240900e-01
   1.32664451e-02]
 [-9.46558522e-01  1.37137534e-02  1.75383025e-04 -1.19767602e-01
   2.83510416e-03  1.02685077e-01 -1.61423631e-01  3.17718409e-03
   1.63445390e-02  1.08423104e-02  2.53327510e-01 -3.34796195e-03
   5.92577320e-01  2.34359635e-02  2.83102161e-02  1.53707137e-03
  -1.65262569e-01 -2.56147663e-01  1.68563641e-02 -6.85926170e-02
   1.16452642e-01 -3.94331438e-03 -2.20989558e-02  1.31773979e-01
   8.19741886e-03]
 [ 1.11147276e-01  3.82841567e-03 -7.54448303e-04  1.96781319e-02
   1.13199623e-02 -3.81419609e-02 -1.63567469e-02  2.89193085e-03
  -1.17193536e-02 -1.13839743e-02 -8.2

In [17]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [18]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))


(53049, 24)
Precision del conjuto de entrenamiento: 58.94%


In [19]:
#Evaluar con el dataset de test: train.csv
# Cargar el dataset
data_t = pd.read_csv('/content/drive/MyDrive/SIS 420 INTELIGENCIA ARTIFICAL 2024/MIS DATASETS/LAB 3/Credit score classification/train.csv', delimiter=',')

dataset_t = data_t.copy()
# Quitamos: numero de seguro social(SSN), identificador(ID) y Customer_ID (Id Cliente) porque no aplica al modelo puesto que no sera un dato
# relavante que determine en nuestro caso
# Eliminar las columna con nombre 'SSN, ID, Customer_ID'
dataset_t.drop('SSN', axis=1, inplace=True)
dataset_t.drop('ID', axis=1, inplace=True)
dataset_t.drop('Customer_ID', axis=1, inplace=True)

# Aqui eliminamos los jemplos que cuenten con campos vacios en alguna de las caracteristicas del dataset.
data_sinf_t = dataset_t.dropna()

# Verificar el nuevo conteo de filas y columnas
print(data_sinf_t.shape)
data_sinf_t.count()
#Tenemos listo el dataset de entrenamiento procesado con datos conscistentes

# Verificar el tipo de datos
print(data_sinf_t.dtypes)

<ipython-input-19-6dedbaaa9ec2>:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data_t = pd.read_csv('/content/drive/MyDrive/SIS 420 INTELIGENCIA ARTIFICAL 2024/MIS DATASETS/LAB 3/Credit score classification/train.csv', delimiter=',')


(53049, 25)
Month                        object
Name                         object
Age                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                 object
dtype: object


In [20]:
#Procedemos a convertir los datos de las caracteristicas a valores numericos para que podamos
#utilizar  de manera eficiente en el modelo mediante pandas
# Convertir columnas categóricas a numéricas (astype('category').cat.codes, convierte las categorías en valores numéricos únicos.)
# Luego, cat.codes extrae estos códigos numéricos asignados a cada categoría.
data_proc_t = data_sinf_t.copy()
data_proc_t['Month'] = data_proc['Month'].astype('category').cat.codes
data_proc_t['Name'] = data_proc['Name'].astype('category').cat.codes
data_proc_t['Age'] = data_proc['Age'].astype('category').cat.codes
data_proc_t['Occupation'] = data_proc['Occupation'].astype('category').cat.codes
data_proc_t['Annual_Income'] = data_proc['Annual_Income'].astype('category').cat.codes
data_proc_t['Num_of_Loan'] = data_proc['Num_of_Loan'].astype('category').cat.codes
data_proc_t['Type_of_Loan'] = data_proc['Type_of_Loan'].astype('category').cat.codes
data_proc_t['Num_of_Delayed_Payment'] = data_proc['Num_of_Delayed_Payment'].astype('category').cat.codes
data_proc_t['Changed_Credit_Limit'] = data_proc['Changed_Credit_Limit'].astype('category').cat.codes
data_proc_t['Credit_Mix'] = data_proc['Credit_Mix'].astype('category').cat.codes
data_proc_t['Outstanding_Debt'] = data_proc['Outstanding_Debt'].astype('category').cat.codes
data_proc_t['Credit_History_Age'] = data_proc['Credit_History_Age'].astype('category').cat.codes
data_proc_t['Payment_of_Min_Amount'] = data_proc['Payment_of_Min_Amount'].astype('category').cat.codes
data_proc_t['Amount_invested_monthly'] = data_proc['Amount_invested_monthly'].astype('category').cat.codes
data_proc_t['Payment_Behaviour'] = data_proc['Payment_Behaviour'].astype('category').cat.codes
data_proc_t['Monthly_Balance'] = data_proc['Monthly_Balance'].astype('category').cat.codes

# Definir el mapeo de clasificación para nuestra y a predecir asignandole los valores que deseamos
clasificacion_t = {'Good': 3, 'Standard': 2, 'Poor': 1}

# Aplicar el mapeo a la columna 'Credit_score'
data_proc_t['Credit_Score'] = data_proc_t['Credit_Score'].map(clasificacion_t)

# Verificar que las columnas categóricas se han convertido a numéricas
print(data_proc_t.head())
print(data_proc_t.dtypes)

#finalmente ordenamos el dataset de manera que la clasificacion del Credit_Score nuestra y a predecir este de manera agrupada y descendente
data_proc_test = data_proc.sort_values(by='Credit_Score')

# Verificar el DataFrame ordenado
print(data_proc_test.head())

    Month  Name  Age  Occupation  Annual_Income  Monthly_Inhand_Salary  \
0       3    75  184          12           4604            1824.843333   
6       4    75  184          12           4604            1824.843333   
8       3  6750  269          15           7905            3037.986667   
9       2  6750  257          13           7905            3037.986667   
10      6  6750  257          13           7906            3037.986667   

    Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  Num_of_Loan  ...  \
0                   3                4              3          134  ...   
6                   3                4              3          134  ...   
8                   2                4              6            1  ...   
9                   2                4              6            1  ...   
10                  2             1385              6            1  ...   

    Credit_Mix  Outstanding_Debt  Credit_Utilization_Ratio  \
0            3             10500          

In [21]:
# La entrada es de 25 caracteristicas + un elemento contando con x0
input_layer_size  = 26

#Etiquetas del Credit_Score (Good=3, Standart=2 y Poor=1)
num_labels = 3

X_test = data_proc_test.iloc[:, :-1]  # Selecciona todas las filas y todas las columnas excepto la última
y_test = data_proc_test.iloc[:, -1]    # Selecciona todas las filas y la última columna

y_test[y_test == 3] = 0
print(X_test)
print(y_test)

m = y_test.size

       Month  Name  Age  Occupation  Annual_Income  Monthly_Inhand_Salary  \
0          3    75  184          12           4604            1824.843333   
35206      4  7728  270           2           9343            3806.948333   
51387      0  6565  385          14           7964            3047.457500   
95414      4  4432  143           3           7478            2729.352817   
95415      1  4432  143           3           7478            2729.352817   
...      ...   ...  ...         ...            ...                    ...   
72508      7  4790  417           5           7639            2999.899167   
72505      2  4790  428           5           7639            2999.899167   
72504      3  4790  417           5           7639            2999.899167   
72519      1  6911  567           3           6610            2273.189167   
93082      6  6724  369          14           9577            4021.730000   

       Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  Num_of_Loan  ... 

<ipython-input-21-fda953f55751>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test[y_test == 3] = 0


In [22]:
# Imprimir la forma de X_test
print(X_test.shape)

#Normalizamos
X_norm, mu, sigma = featureNormalize(X_test)

# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X_test.shape
X_test = X_norm
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

# Realizar predicciones en todo el conjunto de datos X_test
pred = predictOneVsAll(all_theta, X_test)

# Calcular y mostrar la precisión del conjunto de prueba
print('Precisión del conjunto de prueba: {:.2f}%'.format(np.mean(pred == y_test) * 100))

# Imprimir los primeros 10 valores predichos y las etiquetas originales correspondientes del conjunto de prueba
print("Predicciones para los primeros 10 ejemplos:", pred[:10])
print("Etiquetas originales para los primeros 10 ejemplos:", y_test[:10])



(53049, 24)
Precisión del conjunto de prueba: 58.94%
Predicciones para los primeros 10 ejemplos: [2 2 2 2 2 2 2 2 2 2]
Etiquetas originales para los primeros 10 ejemplos: 0        0
35206    0
51387    0
95414    0
95415    0
95416    0
95418    0
35201    0
95420    0
35202    0
Name: Credit_Score, dtype: int64


Una precisión del 58.94% en un problema de clasificación de tres clases no es un resultado gratificante. Hay algunas conclusiones y consideraciones en el metodo realizado que podemos mencionar:

--Se realizaron varios pasos de preprocesamiento de datos, incluyendo la
eliminación de columnas irrelevantes, manejo de valores faltantes y conversión de variables categóricas a numéricas.
La normalización de características también se llevó a cabo para asegurar que todas las características tuvieran un rango similar de valores, lo que puede ayudar a mejorar la convergencia del algoritmo de optimización.

--Probe cambiando el numero de iteraciones en 50, 100, 1000
--Probe cambiando el valor de theta a 0.01 o a 1
 en ambos casos no se consiguio una mejora o resultado diferente.

 Podriamos a llegar a concluir que los datos en el dataset no contienen una relacion que ayuden a realizar un modelo mas preciso.